# **1. Install all the required libraries**

In [ ]:
# installing required libraries

!pip install -U -q pdfplumber tiktoken openai chromaDB sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 706.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.0/383.0 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 604.0/604.0 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#import all the required libaries

import pdfplumber
from pathlib import Path
import pandas as pd
from operator import itemgetter
import json
import tiktoken
import openai
import chromadb
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
#mounting google drive

from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


# 2. Read,Process and Chunck the PDF file



We will be using pdfplumber to read and process the PDF files.

pdfplumber allows for better parsing of the PDF file as it can read various elements of the PDF apart from the plain text, such as, tables, images, etc. It also offers wide functionaties and visual debugging features to help with advanced preprocessing as well.

In [ ]:
#Define the path of the PDF

pdf_path='/content/drive/MyDrive/HelpMateAI/Principal-Sample-Life-Insurance-Policy.pdf'

## 2.1 Reading a single PDF file and exporing it through pdfplumner

In [ ]:
#Open the pdf file
with pdfplumber.open(pdf_path) as pdf:

  #Get on of the pages from the PDF and examine it
  single_page=pdf.pages[16]

  #extract test from first page
  text=single_page.extract_text()

  #extract tables from the page
  tables=single_page.extract_tables()

  #print the extracted text
  print(text)

a. be actively engaged in business for profit within the meaning of the Internal Revenue
Code, or be established as a legitimate nonprofit corporation within the meaning of the
Internal Revenue Code; and
b. make at least the level of premium contributions required for insurance on its eligible
Members. The Policyholder must:
(1) contribute at least 50% of the required premium for all Members (including disabled
Members, if any); and
c. if the Member is to contribute part of the premium, maintain the following participation
percentages with respect to eligible employees and Dependents, excluding those for whom
Proof of Good Health is not satisfactory to The Principal:
(1) Employees:
- at least 75% of all eligible employees must enroll;
(2) Dependents:
- maintain a Dependent participation of at least 75% of eligible Dependents; and
d. if the Member is to contribute no part of the premium, 100% of eligible employees and
Dependents must enroll.
Article 4 - Policy Incontestability
In the ab

In [ ]:
#View the table in the page, if any

tables[4]

IndexError: list index out of range

I am getting error here beacuase it does not contain any table

## 2.2 Extracting text from PDF

In [ ]:
#Function to check wheather a word is present in a table or not for segregation of regular text and tables

def check_bboxes(word,table_bbox):
  #check whether word is inside a table bbox.
  l=word['x0'],word['top'],word['x1'],word['bottom']
  r=table_bbox
  return l[0]>r[0] and l[1]>r[1] and l[2]<r[2] and l[3]<r[3]

In [ ]:
# Function to extract text from a PDF file.
# 1. Declare a variable p to store the iteration of the loop that will help us store page numbers alongside the text
# 2. Declare an empty list 'full_text' to store all the text files
# 3. Use pdfplumber to open the pdf pages one by one
# 4. Find the tables and their locations in the page
# 5. Extract the text from the tables in the variable 'tables'
# 6. Extract the regular words by calling the function check_bboxes() and checking whether words are present in the table or not
# 7. Use the cluster_objects utility to cluster non-table and table words together so that they retain the same chronology as in the original PDF
# 8. Declare an empty list 'lines' to store the page text
# 9. If a text element in present in the cluster, append it to 'lines', else if a table element is present, append the table
# 10. Append the page number and all lines to full_text, and increment 'p'
# 11. When the function has iterated over all pages, return the 'full_text' list

def extract_text_from_pdf(pdf_path):
  p=0
  full_text=[]

  with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
      page_no=f"Page {p+1}"
      text=page.extract_text()

      tables=page.find_tables()
      tables_bboxes=[i.bbox for i in tables]
      tables=[{'table':i.extract(),'top':i.bbox[1]} for i in tables]
      non_table_words=[word for word in page.extract_words() if not any(
          [check_bboxes(word,table_bbox) for table_bbox in tables_bboxes]
      )]
      lines=[]

      for cluster in pdfplumber.utils.cluster_objects(non_table_words + tables,itemgetter('top'),tolerance=5):

        if 'text' in cluster[0]:
          try:
            lines.append(' '.join([i['text'] for i in cluster]))
          except KeyError:
            pass
        elif 'table' in cluster[0]:
          lines.append(json.dumps(cluster[0]['table']))

      full_text.append((page_no,lines))
      p+=1

  return full_text



In [ ]:
#checking extracted text from pdf
extract_text_from_pdf(pdf_path)

[('Page 1',
  ['DOROTHEA GLAUSE S655',
   'RHODE ISLAND JOHN DOE 01/01/2014',
   '711 HIGH STREET',
   'GEORGE RI 02903',
   'GROUP POLICY FOR:',
   'RHODE ISLAND JOHN DOE',
   'ALL MEMBERS',
   'Group Member Life Insurance',
   'Print Date: 07/16/2014']),
 ('Page 2', ['This page left blank intentionally']),
 ('Page 3',
  ['POLICY RIDER',
   'GROUP INSURANCE',
   'POLICY NO: S655',
   'COVERAGE: Life',
   'EMPLOYER: RHODE ISLAND JOHN DOE',
   'Effective on the later of the Date of Issue of this Group Policy or March 1, 2005, the following',
   'will apply to your Policy:',
   'From time to time The Principal may offer or provide certain employer groups who apply',
   'for coverage with The Principal a Financial Services Hotline and Grief Support Services or',
   'any other value added service for the employees of that employer group. In addition, The',
   'Principal may arrange for third party service providers (i.e., optometrists, health clubs), to',
   'provide discounted goods and s

In [ ]:
#Creting table to store data in structure format

extracted_text=extract_text_from_pdf(pdf_path)
pdf_df=pd.DataFrame(extracted_text,columns=['Page No.','Text'])
pdf_df

,Page No.,Text
0,Page 1,"[DOROTHEA GLAUSE S655, RHODE ISLAND JOHN DOE 0..."
1,Page 2,[This page left blank intentionally]
2,Page 3,"[POLICY RIDER, GROUP INSURANCE, POLICY NO: S65..."
3,Page 4,[This page left blank intentionally]
4,Page 5,"[PRINCIPAL LIFE INSURANCE COMPANY, (called The..."
...,...,...
59,Page 60,[I f a Dependent who was insured dies during t...
60,Page 61,"[Section D - Claim Procedures, Article 1 - Not..."
61,Page 62,[A claimant may request an appeal of a claim d...
62,Page 63,[This page left blank intentionally]


In [ ]:
#Checking one of the extracted page texts to ensure that the text has been correctly read
pdf_df.Text[9]

['T he legally recognized union of two eligible individuals of the same sex established according to',
 'law.',
 'Civil Union Partner',
 'For two persons to establish a Civil Union in Rhode Island, it shall be necessary that they satisfy',
 'all of the following criteria:',
 'a. not be a party to another Civil Union or marriage in Rhode Island;',
 'b. be of the same sex and therefore be excluded from the marriage laws of Rhode Island or',
 'any other state;',
 'c. be at least 18 years of age;',
 'd. not be related to the other proposed party to the Civil Union.',
 'NOTE: For the purposes of this Group Policy, the term "spouse" will include Civil Union',
 'Partner, except as otherwise provided in this Group Policy.',
 'Date of Issue',
 'The date this Group Policy is placed in force: November 1, 2007.',
 'Dependent',
 "a. A Member's spouse, if that spouse:",
 '(1) is legally married to the Member; and',
 '(2) is not in the Armed Forces of any country; and',
 '(3) is not insured under thi

In [ ]:
#now let's also check the length of all the texts as there might be aome empty page or pages with very few words that we can drop

pdf_df['Text Length']=pdf_df.Text.apply(lambda x: len(' '.join(x).split(' ')) if isinstance(x,list) else len(x.split(' ')))
pdf_df

,Page No.,Text,Text Length
0,Page 1,"[DOROTHEA GLAUSE S655, RHODE ISLAND JOHN DOE 0...",30
1,Page 2,[This page left blank intentionally],5
2,Page 3,"[POLICY RIDER, GROUP INSURANCE, POLICY NO: S65...",230
3,Page 4,[This page left blank intentionally],5
4,Page 5,"[PRINCIPAL LIFE INSURANCE COMPANY, (called The...",110
...,...,...,...
59,Page 60,[I f a Dependent who was insured dies during t...,285
60,Page 61,"[Section D - Claim Procedures, Article 1 - Not...",418
61,Page 62,[A claimant may request an appeal of a claim d...,322
62,Page 63,[This page left blank intentionally],5


In [ ]:
pdf_df['Text Length']

,Text Length
0,30
1,5
2,230
3,5
4,110
...,...
59,285
60,418
61,322
62,5


In [ ]:
# Store the metadata for each page in a seperate column

pdf_df['Metadata']=pdf_df.apply(lambda x: {'Page No.':x['Page No.']},axis=1)
pdf_df

,Page No.,Text,Text Length,Metadata
0,Page 1,"[DOROTHEA GLAUSE S655, RHODE ISLAND JOHN DOE 0...",30,{'Page No.': 'Page 1'}
1,Page 2,[This page left blank intentionally],5,{'Page No.': 'Page 2'}
2,Page 3,"[POLICY RIDER, GROUP INSURANCE, POLICY NO: S65...",230,{'Page No.': 'Page 3'}
3,Page 4,[This page left blank intentionally],5,{'Page No.': 'Page 4'}
4,Page 5,"[PRINCIPAL LIFE INSURANCE COMPANY, (called The...",110,{'Page No.': 'Page 5'}
...,...,...,...,...
59,Page 60,[I f a Dependent who was insured dies during t...,285,{'Page No.': 'Page 60'}
60,Page 61,"[Section D - Claim Procedures, Article 1 - Not...",418,{'Page No.': 'Page 61'}
61,Page 62,[A claimant may request an appeal of a claim d...,322,{'Page No.': 'Page 62'}
62,Page 63,[This page left blank intentionally],5,{'Page No.': 'Page 63'}


This concludes the chunking aspect also, as we can see that mostly the pages contain few hundred words, maximum going upto 1000. So, we don't need to chunk the documents further; we can perform the embeddings on individual pages. This strategy makes sense for 2 reasons:

1.The way documents are generally
  structured, you will not have a lot of extraneous information in a page, and all the text pieces in that page will likely be interrelated.
  
2.We want to have larger chunk sizes to
  be able to pass appropriate context to the LLM during the generation layer.

# 3.Generate and store Embeddings using OpenAI and ChromaDB

In this section, we will embed the pages in the dataframe through OpenAI's text-embedding-ada-002 model, and store them in a ChromaDB collection.

In [ ]:
# Set the API key
filepath='/content/drive/MyDrive/GenAI_Course_Master/openAIAPIKey.txt'
with open(filepath, 'r') as file:
    api_key = file.read().strip()
    openai.api_key = api_key

In [ ]:
# Importing OpenAI Embedding function into chroma

from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

In [ ]:
# Define the path where chroma collections will be stored

chroma_data_path='/content/drive/MyDrive/HelpMateAI/chromadb_data'

In [ ]:
import chromadb

In [ ]:
#Call PersistentClinet()

client=chromadb.PersistentClient()

In [ ]:
#set up the embedding function using the OpenAI embedding model

model="text-embedding-ada-002"
embedding_function=OpenAIEmbeddingFunction(
    api_key=openai.api_key,
    model_name=model
)

In [ ]:
#Initialise a collection in chroma and pass the embedding_functionto it so that it used OpenAI embeddingd to embed the documents

document_collection=client.get_or_create_collection(
    name='RAG_on_Insurance',
    embedding_function=embedding_function
)

In [ ]:
#Convert the page text and metadata from our dataframe to lists to be able to pass it to chroma

documents_list=pdf_df["Text"].tolist()
metadata_list=pdf_df["Metadata"].tolist()

In [ ]:
#Add the documents and metadata to the collection along with generic integer IDs.We can also feed the metadata information as IDs by combining the policy name and page no.

document_collection.add(
    documents=[str(d) for d in documents_list],
    ids=[str(i) for i in range(0,len(documents_list))],
    metadatas=metadata_list
)

In [ ]:
#let's take a look at the first few entries in the collection

document_collection.get(
    ids=['0','1','2'],
    include=['embeddings','documents','metadatas']
)

{'ids': ['0', '1', '2'],
 'embeddings': array([[-0.03006636,  0.02741426, -0.01578896, ..., -0.0319352 ,
         -0.0085472 , -0.01249788],
        [-0.02222201, -0.00129016, -0.00024185, ..., -0.00927748,
          0.00503704, -0.02812774],
        [-0.01564728,  0.01715613, -0.00729974, ..., -0.01913998,
         -0.00207466, -0.00375116]]),
 'metadatas': [{'Page No.': 'Page 1'},
  {'Page No.': 'Page 2'},
  {'Page No.': 'Page 3'}],
 'documents': ["['DOROTHEA GLAUSE S655', 'RHODE ISLAND JOHN DOE 01/01/2014', '711 HIGH STREET', 'GEORGE RI 02903', 'GROUP POLICY FOR:', 'RHODE ISLAND JOHN DOE', 'ALL MEMBERS', 'Group Member Life Insurance', 'Print Date: 07/16/2014']",
  "['This page left blank intentionally']",
  "['POLICY RIDER', 'GROUP INSURANCE', 'POLICY NO: S655', 'COVERAGE: Life', 'EMPLOYER: RHODE ISLAND JOHN DOE', 'Effective on the later of the Date of Issue of this Group Policy or March 1, 2005, the following', 'will apply to your Policy:', 'From time to time The Principal may offe

In [ ]:
cache_collection=client.get_or_create_collection(name='document_Cache',embedding_function=embedding_function)

In [ ]:
cache_collection.peek()

{'ids': [],
 'embeddings': array([], dtype=float64),
 'metadatas': [],
 'documents': [],
 'uris': None,
 'data': None,
 'included': ['embeddings', 'metadatas', 'documents']}

# 4.Semantic Search with Cache

In [ ]:
#Read the user query
query=input()

what does this document refers to?


In [ ]:
#search the cache collection first
#query the collection against the user query and return the top 20 results

cache_results=cache_collection.query(
    query_texts=query,
    n_results=1,
)

In [ ]:
cache_results

{'ids': [[]],
 'distances': [[]],
 'metadatas': [[]],
 'embeddings': None,
 'documents': [[]],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}

In [ ]:
results=document_collection.query(
    query_texts=query,
    n_results=10,
)


In [ ]:
#Implementing cache in semantic search

#set a threshold for cache search
threshold=0.2

ids=[]
documents=[]
distances=[]
metadatas=[]
results_df=pd.DataFrame()

# If the distance is greater than the threshold, then return the results from the main collection.

if cache_results['distances'][0] == [] or cache_results['distances'][0][0] > threshold:
      # Query the collection against the user query and return the top 10 results
      results = document_collection.query(
      query_texts=query,
      n_results=10
      )

      # Store the query in cache_collection as document w.r.t to ChromaDB so that it can be embedded and searched against later
      # Store retrieved text, ids, distances and metadatas in cache_collection as metadatas, so that they can be fetched easily if a query indeed matches to a query in cache
      Keys = []
      Values = []

      for key, val in results.items():
        if val is None:
          continue
        for i in range(len(val[0])):
          Keys.append(str(key)+str(i))
          Values.append(str(val[0][i]))


      cache_collection.add(
          documents= [query],
          ids = [query],  # Or if you want to assign integers as IDs 0,1,2,.., then you can use "len(cache_results['documents'])" as will return the no. of queries currently in the cache and assign the next digit to the new query."
          metadatas = dict(zip(Keys, Values))
      )

      print("Not found in cache. Found in main collection.")

      result_dict = {'Metadatas': results['metadatas'][0], 'Documents': results['documents'][0], 'Distances': results['distances'][0], "IDs":results["ids"][0]}
      results_df = pd.DataFrame.from_dict(result_dict)
      results_df


# If the distance is, however, less than the threshold, you can return the results from cache

elif cache_results['distances'][0][0] <= threshold:
      cache_result_dict = cache_results['metadatas'][0][0]

      # Loop through each inner list and then through the dictionary
      for key, value in cache_result_dict.items():
          if 'ids' in key:
              ids.append(value)
          elif 'documents' in key:
              documents.append(value)
          elif 'distances' in key:
              distances.append(value)
          elif 'metadatas' in key:
              metadatas.append(value)

      print("Found in cache!")

      # Create a DataFrame
      results_df = pd.DataFrame({
        'IDs': ids,
        'Documents': documents,
        'Distances': distances,
        'Metadatas': metadatas
      })

Not found in cache. Found in main collection.


In [ ]:
results_df

,Metadatas,Documents,Distances,IDs
0,{'Page No.': 'Page 15'},['A record which is on or transmitted by paper...,0.414437,14
1,{'Page No.': 'Page 14'},"['c . end stage renal failure; or', 'd. acquir...",0.445623,13
2,{'Page No.': 'Page 18'},"[""c . a copy of the form which contains the st...",0.447462,17
3,{'Page No.': 'Page 6'},"['TABLE OF CONTENTS', 'PART I - DEFINITIONS', ...",0.454058,5
4,{'Page No.': 'Page 4'},['This page left blank intentionally'],0.458574,3
5,{'Page No.': 'Page 2'},['This page left blank intentionally'],0.458574,1
6,{'Page No.': 'Page 63'},['This page left blank intentionally'],0.458574,62
7,{'Page No.': 'Page 16'},"['PART II - POLICY ADMINISTRATION', 'Section A...",0.465076,15
8,{'Page No.': 'Page 19'},['T he Principal has complete discretion to co...,0.467052,18
9,{'Page No.': 'Page 37'},"['b. a business assignment; or', 'c. full-time...",0.467728,36


## 5. Re-Ranking with a Cross Encoder


Re-ranking the results obtained from our semantic search can sometime significantly improve the relevance of the retrieved results. This is often done by passing the query paired with each of the retrieved responses into a cross-encoder to score the relevance of the response w.r.t. the query.

In [ ]:
#Import the CrossEncoder library from sentence_transformers

from sentence_transformers import CrossEncoder,util


In [ ]:
#Initialize the cross encoder model

cross_encoder=CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
#Test the cross encoder model

scores=cross_encoder.predict([
    ['What is the resons required for a person to be qualified as total disable?'],
    ['What are the Principal reserves the right to change this Group Policy']])

In [ ]:
scores

array([-6.7822604, -9.1800995], dtype=float32)

In [ ]:
#Input (query,response) pairs fro each of the top 20 responses received from the semantic search to the cross encoder
#Generate the cross_encoder scores for these pairs

cross_inputs=[[query,response] for response in results_df['Documents']]
cross_rerank_scores=cross_encoder.predict(cross_inputs)

In [ ]:
cross_rerank_scores

array([ -3.6882458,  -7.15013  ,  -7.0768557,  -9.999155 , -10.117714 ,
       -10.117714 , -10.117714 ,  -8.061248 ,  -8.269232 ,  -9.319514 ],
      dtype=float32)

In [ ]:
#Store the rerank_scores in results_df
results_df['Reranked_scores']=cross_rerank_scores

In [ ]:
results_df

,Metadatas,Documents,Distances,IDs,Reranked_scores
0,{'Page No.': 'Page 15'},['A record which is on or transmitted by paper...,0.414437,14,-3.688246
1,{'Page No.': 'Page 14'},"['c . end stage renal failure; or', 'd. acquir...",0.445623,13,-7.150130
2,{'Page No.': 'Page 18'},"[""c . a copy of the form which contains the st...",0.447462,17,-7.076856
3,{'Page No.': 'Page 6'},"['TABLE OF CONTENTS', 'PART I - DEFINITIONS', ...",0.454058,5,-9.999155
4,{'Page No.': 'Page 4'},['This page left blank intentionally'],0.458574,3,-10.117714
5,{'Page No.': 'Page 2'},['This page left blank intentionally'],0.458574,1,-10.117714
6,{'Page No.': 'Page 63'},['This page left blank intentionally'],0.458574,62,-10.117714
7,{'Page No.': 'Page 16'},"['PART II - POLICY ADMINISTRATION', 'Section A...",0.465076,15,-8.061248
8,{'Page No.': 'Page 19'},['T he Principal has complete discretion to co...,0.467052,18,-8.269232
9,{'Page No.': 'Page 37'},"['b. a business assignment; or', 'c. full-time...",0.467728,36,-9.319514


In [ ]:
#Return the top 3 results from semantic search

top_3_semantic=results_df.sort_values(by='Distances')
top_3_semantic[:3]

,Metadatas,Documents,Distances,IDs,Reranked_scores
0,{'Page No.': 'Page 15'},['A record which is on or transmitted by paper...,0.414437,14,-3.688246
1,{'Page No.': 'Page 14'},"['c . end stage renal failure; or', 'd. acquir...",0.445623,13,-7.150130
2,{'Page No.': 'Page 18'},"[""c . a copy of the form which contains the st...",0.447462,17,-7.076856
